In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler as minmax


In [2]:
df = pd.read_json('dataset/train')
Y = df.is_iceberg.as_matrix()
del df['is_iceberg']
del df['id']
band1 = df['band_1'].as_matrix()
band2 = df['band_2'].as_matrix()
angle = df['inc_angle'].as_matrix()
df.head()

,band_1,band_2,inc_angle
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256


In [3]:

band1 = np.asarray(band1.tolist()).reshape(1604*75*75,-1)
band2 = np.asarray(band1.tolist()).reshape(1604*75*75,-1)
scaler = minmax(feature_range = (0,1))

scaler.fit(band1)
band1 = scaler.transform(band1).reshape(-1,1)

scaler.fit(band2)
band2 = scaler.transform(band2).reshape(-1,1)


In [4]:

band1 = band1.reshape(1604,75,75)
band2 = band2.reshape(1604,75,75)

In [5]:
X = np.stack((band1,band2),axis=3)
X.shape

(1604, 75, 75, 2)

In [18]:

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=(75, 75,2)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [20]:
model.fit(X,Y,epochs=15)

Epoch 1/15
1604/1604 [==============================] - 17s - loss: 0.5546 - acc: 0.7020    
Epoch 2/15
1604/1604 [==============================] - 19s - loss: 0.5252 - acc: 0.7425    
Epoch 3/15
1604/1604 [==============================] - 19s - loss: 0.5075 - acc: 0.7350    
Epoch 4/15
1604/1604 [==============================] - 20s - loss: 0.5020 - acc: 0.7500    
Epoch 5/15
1604/1604 [==============================] - 20s - loss: 0.4735 - acc: 0.7743    
Epoch 6/15
1604/1604 [==============================] - 20s - loss: 0.4437 - acc: 0.8042    
Epoch 7/15
1604/1604 [==============================] - 21s - loss: 0.4368 - acc: 0.7955    
Epoch 8/15
1604/1604 [==============================] - 20s - loss: 0.4497 - acc: 0.7911    
Epoch 9/15
1604/1604 [==============================] - 19s - loss: 0.4103 - acc: 0.8105    
Epoch 10/15
1604/1604 [==============================] - 19s - loss: 0.3988 - acc: 0.8111    
Epoch 11/15
1604/1604 [==============================] - 20s - loss: 

In [24]:
# serialize model to JSON
model_json = model.to_json()
with open("savedModels/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("savedModels/model.h5")
print("Saved model to disk")

Saved model to disk


In [7]:

# load json and create model
json_file = open('savedModels/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("savedModels/model.h5")
print("Loaded model from disk")

ImportError: `load_weights` requires h5py.